In [62]:
pd.options.mode.chained_assignment = None
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

In [12]:
labels = train_data['label']
train = pd.concat([train_data.iloc[:, 0:200], train_data.iloc[:, 368:len(train_data.columns)]], axis=1)
test = pd.concat([test_data.iloc[:, 0:200], test_data.iloc[:, 368:len(test_data.columns)]], axis=1)

In [13]:
train_without_nans = train_data.iloc[:, 0:200]

for i in range(0, 200):
    col_name = 'raw_' + str(i)
    
    y = train_data[~train_data[col_name].isnull()][col_name]
    X = train_data[~train_data[col_name].isnull()].iloc[:, 344:368]

    regressor = LinearRegression()  
    regressor.fit(X, y)
    
    y_pred = pd.DataFrame(regressor.predict(train_data.iloc[:, 344:368])).iloc[:, 0]
    train_without_nans[col_name] = train_without_nans[col_name].fillna(y_pred.round())

In [63]:
rfc = RandomForestClassifier(n_estimators=200,max_features=30)
rfc.fit(train_without_nans, labels)

KeyboardInterrupt: 

In [55]:
results = pd.DataFrame(rfc.predict(test_data.iloc[:, 0:200]))

In [23]:
# fit model no training data
model = XGBClassifier()
model.fit(train_without_nans, labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [27]:
# make predictions for test data
y_pred = pd.DataFrame(model.predict(test_data.iloc[:, 0:200]))

In [36]:
ada = AdaBoostClassifier()
ada.fit(train_without_nans,labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [37]:
ada_pred = pd.DataFrame(ada.predict(test_data.iloc[:, 0:200]))

In [58]:
results

,0
0,4
1,8
2,6
3,4
4,8
5,3
6,5
7,6
8,2
9,4


In [28]:
# y = train.pop('label')
clf = RandomForestClassifier(n_jobs=2)
clf.fit(train, y)

D:\conda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
result = clf.predict(test)

In [30]:
result[:50]

array([0., 0., 0., 2., 8., 1., 3., 6., 2., 1., 6., 9., 0., 0., 0., 2., 2.,
       0., 3., 4., 5., 4., 7., 1., 1., 3., 1., 7., 9., 5., 4., 7., 7., 6.,
       4., 7., 9., 3., 3., 5., 0., 0., 0., 6., 4., 2., 0., 9., 4., 0.])

In [59]:
sample = pd.read_csv('sample.csv')
sample['label'] = results

In [60]:
sample.to_csv('output_rf.csv',index=False)